<h1>Ensemble model</h1>


Version 1, 17 Jun 2016<br>
Jan Šnajder

In [482]:
import sys

import scipy as sp
import pandas as pd

from composes.matrix.dense_matrix import DenseMatrix
from composes.matrix.sparse_matrix import SparseMatrix
from composes.semantic_space.space import Space
from composes.similarity.cos import CosSimilarity
from composes.similarity.similarity import Similarity
from composes.utils import io_utils
from composes.transformation.scaling.row_normalization import RowNormalization

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [483]:
proj_path = '/home/jan/b9/derivsem/'
src_path = proj_path + 'src/'
data_path = '/data/dsm/sdewac/'
sys.path.append(src_path)

In [485]:
from Models import *

In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Distributional spaces

In [93]:
space = {}

### CBOW

In [116]:
model_file = 'cbow/cbow_300dim_hs0/sdewac.300.cbow.hs0.w2.vsm.pkl'
space['cbow-w2'] = io_utils.load(data_path + model_file).apply(RowNormalization(criterion = 'length'))
model_file = 'cbow/cbow_300dim_hs0/sdewac.300.cbow.hs0.w5.vsm.pkl'
space['cbow-w5'] = io_utils.load(data_path + model_file).apply(RowNormalization(criterion = 'length'))
model_file = 'cbow/cbow_300dim_hs0/sdewac.300.cbow.hs0.w10.vsm.pkl'
space['cbow-w10'] = io_utils.load(data_path + model_file).apply(RowNormalization(criterion = 'length'))

space_cbow = space['cbow-w2']

In [95]:
space_cbow.cooccurrence_matrix

In [62]:
type(space_cbow.cooccurrence_matrix)

composes.matrix.sparse_matrix.SparseMatrix

In [11]:
isinstance(space_cbow.cooccurrence_matrix, DenseMatrix)

True

In [41]:
sp.shape(space_cbow.cooccurrence_matrix)

(289699, 300)

### Count-based

In [117]:
model_file = 'count-based/sdewac_2015-11-23/sdewac-mst.prepro.bow-c10k-w5.ppmi.matrix.pkl'
space['ppmi'] = io_utils.load(data_path + model_file).apply(RowNormalization(criterion = 'length'))

In [78]:
space['ppmi'].cooccurrence_matrix

In [79]:
type(space['ppmi'].cooccurrence_matrix)

composes.matrix.sparse_matrix.SparseMatrix

In [80]:
sp.shape(space['ppmi'].cooccurrence_matrix)

(289393, 10000)

### TODO: COW model

### Gur350 check

In [53]:
gur350_gold = sp.genfromtxt(proj_path + "data/gur350-gold.txt", dtype=None, names=('w1', 'w2', 'gold', 'stdev'))

In [106]:
%%capture
predicted_cbow = space_cbow.get_sims(gur350_gold[['w1','w2']], CosSimilarity());

In [107]:
eval_correlation(predicted_cbow, gur350_gold['gold'])

'r = 0.469658, rho = 0.429867, r_cov = 0.655112, rho_cov = 0.686132, cov = 293 (0.84%)'

In [56]:
%%capture
predicted_ppmi = space_ppmi.get_sims(gur350_gold[['w1','w2']], CosSimilarity());

In [57]:
evalCorrelation(predicted_ppmi, gur350_gold['gold'])

'r = 0.469658, rho = 0.429867, r_cov = 0.655112, rho_cov = 0.686132, cov = 293 (0.84%)'

In [110]:
space.keys()

['cbow-w5', 'ppmi', 'cbow-w10', 'cbow-w2']

In [118]:
%%capture
pred = {}
for name in space.keys():
    pred[name] = space[name].get_sims(gur350_gold[['w1', 'w2']], CosSimilarity());

In [119]:
for name in sorted(space.keys()):
    print('%s: %s' % (name, evalCorrelation(pred[name], gur350_gold['gold'])))

cbow-w10: r = 0.590681, rho = 0.572978, r_cov = 0.740932, rho_cov = 0.745762, cov = 240 (0.69%)
cbow-w2: r = 0.603446, rho = 0.570147, r_cov = 0.769273, rho_cov = 0.782691, cov = 260 (0.74%)
cbow-w5: r = 0.598424, rho = 0.570678, r_cov = 0.754332, rho_cov = 0.759574, cov = 248 (0.71%)
ppmi: r = 0.469658, rho = 0.429867, r_cov = 0.655112, rho_cov = 0.686132, cov = 293 (0.84%)


# Data

In [429]:
pairs_df = pd.read_csv(proj_path + "data/pairs-all.txt", sep=' ')

In [430]:
pairs_df

,pattern,word1,word2,polysemy,invCL
0,dAA02,abgedeckt_A,unabgedeckt_A,0,0.051396
1,dAA02,abhängig_A,unabhängig_A,3,0.494741
2,dAA02,abkömmlich_A,unabkömmlich_A,0,0.354503
3,dAA02,absehbar_A,unabsehbar_A,1,0.214439
4,dAA02,absetzbar_A,unabsetzbar_A,2,0.047764
5,dAA02,absichtlich_A,unabsichtlich_A,1,0.245723
6,dAA02,abwendbar_A,unabwendbar_A,0,0.517306
7,dAA02,achtsam_A,unachtsam_A,1,0.353418
8,dAA02,ähnlich_A,unähnlich_A,1,0.068213
9,dAA02,anfechtbar_A,unanfechtbar_A,1,0.502672


In [322]:
pairs_df.groupby('pattern').size().sort_values(ascending=False)

pattern
dVA02    3028
dVA13    2442
dVN07    2324
dNA26    1323
dAN03    1192
dNA25    1174
dVN03     936
dVN09     882
dAA02     635
dNV09     590
dNV17     530
dVA12     476
dAN02     454
dNA05     453
dNA29     438
dVV05     428
dVV26     390
dNA02     389
dVN10     367
dVV13     340
dAN04     330
dVV22     328
dNV22     304
dVA01     294
dVV14     291
dVV02     269
dVA03     255
dNV20     232
dVV30     225
dVV31     199
dNA01     196
dAA15     193
dVV29     190
dAN16     186
dAV04     185
dVA11     180
dNV11     176
dNV01     170
dVV33     167
dVV03     162
dAN11     160
dVN04     159
dNA27     157
dNV02     149
dNV05     143
dAN09     136
dNV24     130
dAV06     122
dNA06     113
dAA03     112
dAN15     109
dAV01     105
dNA03     103
dAN12      98
dVV01      91
dNV08      91
dVV25      87
dVV34      86
dAN10      83
dtype: int64

In [431]:
def partition_pairs(pairs_df, split, random_state=None):
    df = pd.DataFrame()
    ps = []
    for _, p in pairs_df.groupby('pattern'):
        n = len(p)
        ix = index_partitions(n, split, random_state=random_state)
        p.loc[:,'partition'] = pd.Series(ix, index=p.index)
        ps.append(p)
    return pd.concat(ps)

In [433]:
split = [0.5, 0.3, 0.2]
partitioned_pairs_df = partition_pairs(pairs_df, split, random_state=42)

In [434]:
partitioned_pairs_df

,pattern,word1,word2,polysemy,invCL,partition
0,dAA02,abgedeckt_A,unabgedeckt_A,0,0.051396,1
1,dAA02,abhängig_A,unabhängig_A,3,0.494741,0
2,dAA02,abkömmlich_A,unabkömmlich_A,0,0.354503,1
3,dAA02,absehbar_A,unabsehbar_A,1,0.214439,0
4,dAA02,absetzbar_A,unabsetzbar_A,2,0.047764,0
5,dAA02,absichtlich_A,unabsichtlich_A,1,0.245723,1
6,dAA02,abwendbar_A,unabwendbar_A,0,0.517306,1
7,dAA02,achtsam_A,unachtsam_A,1,0.353418,2
8,dAA02,ähnlich_A,unähnlich_A,1,0.068213,2
9,dAA02,anfechtbar_A,unanfechtbar_A,1,0.502672,0


In [435]:
partitioned_pairs_df.groupby(['pattern', 'partition']).size().unstack(level=-1)

partition,0,1,2
pattern,,,
dAA02,318,190,127
dAA03,56,34,22
dAA15,97,57,39
dAN02,227,136,91
dAN03,596,358,238
dAN04,165,99,66
dAN09,68,41,27
dAN10,42,24,17
dAN11,80,48,32


In [453]:
def filter_pairs(pairs_df, pattern, partition=None):
    ix = pairs_df['pattern'] == pattern
    if partition is not None:
        ix &= pairs_df['partition'] == partition
    return pairs_df[ix]

In [470]:
def get_word_pairs(pairs_df):
    return pairs_df[['word1', 'word2']].values.tolist()

# Model predictions

In [403]:
model = {}
for s in space.keys():
    model['baseline-' + s] = BaselineModel(space[s])
    model['add-' + s] = AdditiveModel(space[s])
    model['lexfun-' + s] = LexfunModel(space[s], learner='Ridge')

In [404]:
patterns = list(sorted(set(pairs_df['pattern'])))

def pattern_pos(pattern): return pattern[1], pattern[2]

In [480]:
def evaluate(partitioned_pairs_df, models_dict, patterns=None):
    if patterns is not None:
        partitioned_pairs_df = partitioned_pairs_df[partitioned_pairs_df['pattern'].isin(patterns)]
    for model_name, model in models_dict.items():
        dfs = []
        for pattern, pairs_df in partitioned_pairs_df.groupby('pattern'):
            print model_name, pattern
            train_pairs = get_word_pairs(filter_pairs(pairs_df, pattern, 0))
            model.fit(train_pairs)
            _, target_pos = pattern_pos(pattern)
            scores_test = reciprocal_rank_scores(model, get_word_pairs(pairs_df), pos=target_pos)
            pairs_df.loc[:,'baseline-cbow-w5'] = pd.Series(scores_test, index=pairs_df.index)
            dfs.append(pairs_df)
        return pd.concat(dfs)

In [481]:
evaluate(pairs_df, {'baseline' : BaselineModel(space['cbow-w5'])}, patterns=['dAA03'])

baseline dAA03


,pattern,word1,word2,polysemy,invCL,partition,baseline-cbow-w5
635,dAA03,albanisch_A,antialbanisch_A,1,0.044741,0,0.011905
636,dAA03,alkoholisch_A,antialkoholisch_A,1,0.044674,1,0.200000
637,dAA03,amerikanisch_A,antiamerikanisch_A,1,0.077198,0,0.010526
638,dAA03,arabisch_A,antiarabisch_A,1,0.038823,1,0.023810
639,dAA03,armenisch_A,antiarmenisch_A,1,0.016906,0,0.025000
640,dAA03,aufklärerisch_A,antiaufklärerisch_A,2,0.122072,1,0.500000
641,dAA03,autoritär_A,antiautoritär_A,1,0.212629,1,0.035714
642,dAA03,bakteriell_A,antibakteriell_A,1,0.193392,2,0.034483
643,dAA03,biotisch_A,antibiotisch_A,0,0.236097,0,0.000584
644,dAA03,britisch_A,antibritisch_A,1,0.030796,1,0.004202


# Combine predictions

In [564]:
pairs_df = pd.read_csv(proj_path + "data/pairs-all.txt", sep=' ')
df = pd.read_pickle(proj_path + "results/baseline-cbow-w2.pkl")
df0 = pd.concat([pairs_df, df['partition']], axis=1)

In [565]:
models = ['baseline', 'add', 'lexfun']
spaces = ['cbow-w2', 'cbow-w5', 'cbow-w10']
dfs = [df0]
for i, m in enumerate(models):
    for j, s in enumerate(spaces):
        name = '%s-%s' % (m, s)
        df = pd.read_pickle('%s/results/%s.pkl' % (proj_path, name)).rename(columns = {'baseline-cbow-w5': name})#.set_index(pairs_df.index)
        dfs.append(df[name])
df = pd.concat(dfs, axis=1)
df

,pattern,word1,word2,polysemy,invCL,partition,baseline-cbow-w2,baseline-cbow-w5,baseline-cbow-w10,add-cbow-w2,add-cbow-w5,add-cbow-w10,lexfun-cbow-w2,lexfun-cbow-w5,lexfun-cbow-w10
0,dAA02,abgedeckt_A,unabgedeckt_A,0,0.051396,1,0.166667,0.023256,0.000891,0.166667,0.016129,0.000734,0.001558,0.001988,0.000134
1,dAA02,abhängig_A,unabhängig_A,3,0.494741,0,0.500000,0.250000,0.500000,0.500000,0.200000,0.500000,1.000000,1.000000,1.000000
2,dAA02,abkömmlich_A,unabkömmlich_A,0,0.354503,1,0.010526,0.012821,0.071429,0.004878,0.013333,0.058824,0.000100,0.000200,0.013699
3,dAA02,absehbar_A,unabsehbar_A,1,0.214439,0,0.200000,0.090909,0.029412,0.333333,0.333333,0.083333,1.000000,1.000000,0.500000
4,dAA02,absetzbar_A,unabsetzbar_A,2,0.047764,0,0.001698,0.003390,0.000216,0.002506,0.005747,0.000276,1.000000,1.000000,1.000000
5,dAA02,absichtlich_A,unabsichtlich_A,1,0.245723,1,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.333333,0.333333,0.500000
6,dAA02,abwendbar_A,unabwendbar_A,0,0.517306,1,0.200000,0.200000,0.166667,0.200000,0.250000,0.250000,0.058824,0.035714,0.022727
7,dAA02,achtsam_A,unachtsam_A,1,0.353418,2,0.012048,0.004348,0.002577,0.055556,0.016667,0.006579,0.055556,0.035714,0.012500
8,dAA02,ähnlich_A,unähnlich_A,1,0.068213,2,0.166667,0.142857,0.031250,0.166667,0.142857,0.035714,0.000357,0.001236,0.000923
9,dAA02,anfechtbar_A,unanfechtbar_A,1,0.502672,0,0.500000,0.333333,0.333333,0.500000,0.333333,0.500000,1.000000,1.000000,1.000000


In [541]:
writer = pd.ExcelWriter(proj_path + '/results/b9-ensemble-results-1.xlsx')
df.to_excel(writer,'Base models')
writer.save()

In [632]:
partitioned_pairs_df = df[['pattern', 'word1', 'word2', 'partition']]
partitioned_pairs_df.to_csv(proj_path + 'data/partitioned-pairs.csv')
len(partitioned_pairs_df)

25557

In [634]:
partitioned_pairs_df = pd.read_csv(proj_path + "data/partitioned-pairs.csv", sep=',', index_col=0)
pairs_df = partitioned_pairs_df.drop('partition', axis=1)
partitioned_pairs_df

,pattern,word1,word2,partition
0,dAA02,abgedeckt_A,unabgedeckt_A,1
1,dAA02,abhängig_A,unabhängig_A,0
2,dAA02,abkömmlich_A,unabkömmlich_A,1
3,dAA02,absehbar_A,unabsehbar_A,0
4,dAA02,absetzbar_A,unabsetzbar_A,0
5,dAA02,absichtlich_A,unabsichtlich_A,1
6,dAA02,abwendbar_A,unabwendbar_A,1
7,dAA02,achtsam_A,unachtsam_A,2
8,dAA02,ähnlich_A,unähnlich_A,2
9,dAA02,anfechtbar_A,unanfechtbar_A,0


In [595]:
invcl_df = df[['pattern','word1','word2','invCL']]
rr_df = df[['pattern','word1','word2','baseline-cbow-w2', 'baseline-cbow-w5', 'baseline-cbow-w10', 'add-cbow-w2', 'add-cbow-w5', 'add-cbow-w10', 'lexfun-cbow-w2', 'lexfun-cbow-w5', 'lexfun-cbow-w10']]

In [609]:
wn_df = pd.read_csv(proj_path + "tmp/WN/wn-pairs.txt", sep=' ')
deriv_df = pd.read_csv(proj_path + "tmp/binDeriv/binDeriv-pairs.txt", sep=' ')

In [685]:
pos1 = pairs_df.apply(lambda x : pattern_pos(x['pattern'])[0] , axis=1).rename('pos1')
pos2 = pairs_df.apply(lambda x : pattern_pos(x['pattern'])[1] , axis=1).rename('pos2')
pos_df = pd.concat([pairs_df, pos1, pos2], axis=1)

In [ ]:
freq_df = pd.read_csv(proj_path + "data/sdewac-mst.prepro.lemmas", sep='\t', header=None, names=['lemma','freq']).set_index('lemma')
freq_dict = freq_df['freq'].to_dict()

In [690]:
freq1 = pairs_df.apply(lambda x : freq_dict.get(x['word1'], 0), axis=1).rename('freq1')
freq2 = pairs_df.apply(lambda x : freq_dict.get(x['word2'], 0), axis=1).rename('freq2')
freq_df = pd.concat([pairs_df, freq1, freq2], axis=1)

In [696]:
num_pairs = partitioned_pairs_df.groupby(['pattern', 'partition']).size()
def num_pattern_pairs(pattern, partition):
    return num_pairs[pattern][partition]

In [697]:
num_pattern_pairs('dAA02', 1)

190

In [699]:
num_pairs0 = pairs_df.apply(lambda x : num_pattern_pairs(x['pattern'], 0), axis=1).rename('num_pairs0')
num_pairs1 = pairs_df.apply(lambda x : num_pattern_pairs(x['pattern'], 1), axis=1).rename('num_pairs1')
num_pairs2 = pairs_df.apply(lambda x : num_pattern_pairs(x['pattern'], 2), axis=1).rename('num_pairs2')
num_pairs_df = pd.concat([pairs_df, num_pairs0, num_pairs1, num_pairs2], axis=1)

In [753]:
pairs_features_df = partitioned_pairs_df.merge(pos_df).merge(deriv_df).merge(num_pairs_df).merge(wn_df).merge(freq_df).merge(invcl_df);
pairs_features_df

,pattern,word1,word2,partition,pos1,pos2,deriv,num_pairs0,num_pairs1,num_pairs2,polysemy1,polysemy2,freq1,freq2,invCL
0,dAA02,abgedeckt_A,unabgedeckt_A,1,A,A,1,318,190,127,0,0,536,8,0.051396
1,dAA02,abhängig_A,unabhängig_A,0,A,A,1,318,190,127,3,3,42253,85638,0.494741
2,dAA02,abkömmlich_A,unabkömmlich_A,1,A,A,0,318,190,127,0,1,67,181,0.354503
3,dAA02,absehbar_A,unabsehbar_A,0,A,A,1,318,190,127,1,1,13552,1539,0.214439
4,dAA02,absetzbar_A,unabsetzbar_A,0,A,A,1,318,190,127,2,0,1172,38,0.047764
5,dAA02,absichtlich_A,unabsichtlich_A,1,A,A,1,318,190,127,1,1,6111,840,0.245723
6,dAA02,abwendbar_A,unabwendbar_A,1,A,A,1,318,190,127,0,1,177,1140,0.517306
7,dAA02,achtsam_A,unachtsam_A,2,A,A,1,318,190,127,1,1,620,449,0.353418
8,dAA02,ähnlich_A,unähnlich_A,2,A,A,0,318,190,127,1,1,130027,895,0.068213
9,dAA02,anfechtbar_A,unanfechtbar_A,0,A,A,1,318,190,127,1,1,1139,1347,0.502672


In [758]:
pairs_features_df.to_csv(proj_path + 'data/pairs-features.csv')

# Factor out wrt distributional space

In [755]:
dfs = []
for i, w in enumerate([2, 5, 10]):
    pred_df = rr_df[['baseline-cbow-w%s' % w, 'add-cbow-w%s' % w, 'lexfun-cbow-w%s' % w]].copy()
    pred_df.columns = ['rr_baseline', 'rr_add', 'rr_lexfun']
    pred_df['space'] = 'cbow'
    pred_df['window'] = w
    pred_df = pred_df.reindex(columns = ['space', 'window', 'rr_baseline', 'rr_add', 'rr_lexfun'])
    df = pd.concat([pairs_features_df, pred_df], axis=1)
    dfs.append(df)
pairs_features_rr_df = pd.concat(dfs).sort_values(['pattern', 'word1', 'word2'])
pairs_features_rr_df

,pattern,word1,word2,partition,pos1,pos2,deriv,num_pairs0,num_pairs1,num_pairs2,polysemy1,polysemy2,freq1,freq2,invCL,space,window,rr_baseline,rr_add,rr_lexfun
0,dAA02,abgedeckt_A,unabgedeckt_A,1,A,A,1,318,190,127,0,0,536,8,0.051396,cbow,2,0.166667,0.166667,0.001558
0,dAA02,abgedeckt_A,unabgedeckt_A,1,A,A,1,318,190,127,0,0,536,8,0.051396,cbow,5,0.023256,0.016129,0.001988
0,dAA02,abgedeckt_A,unabgedeckt_A,1,A,A,1,318,190,127,0,0,536,8,0.051396,cbow,10,0.000891,0.000734,0.000134
1,dAA02,abhängig_A,unabhängig_A,0,A,A,1,318,190,127,3,3,42253,85638,0.494741,cbow,2,0.500000,0.500000,1.000000
1,dAA02,abhängig_A,unabhängig_A,0,A,A,1,318,190,127,3,3,42253,85638,0.494741,cbow,5,0.250000,0.200000,1.000000
1,dAA02,abhängig_A,unabhängig_A,0,A,A,1,318,190,127,3,3,42253,85638,0.494741,cbow,10,0.500000,0.500000,1.000000
2,dAA02,abkömmlich_A,unabkömmlich_A,1,A,A,0,318,190,127,0,1,67,181,0.354503,cbow,2,0.010526,0.004878,0.000100
2,dAA02,abkömmlich_A,unabkömmlich_A,1,A,A,0,318,190,127,0,1,67,181,0.354503,cbow,5,0.012821,0.013333,0.000200
2,dAA02,abkömmlich_A,unabkömmlich_A,1,A,A,0,318,190,127,0,1,67,181,0.354503,cbow,10,0.071429,0.058824,0.013699
3,dAA02,absehbar_A,unabsehbar_A,0,A,A,1,318,190,127,1,1,13552,1539,0.214439,cbow,2,0.200000,0.333333,1.000000


In [759]:
pairs_features_rr_df.to_csv(proj_path + 'data/pairs-features-rr.csv')

All together in a single xlsx

In [762]:
patterns_df = partitioned_pairs_df.groupby(['pattern', 'partition']).size().unstack(level=-1)

In [763]:
writer = pd.ExcelWriter(proj_path + '/data/b9-data-v1.xlsx')
patterns_df.to_excel(writer,'patterns')
partitioned_pairs_df.to_excel(writer,'pairs')
pairs_features_df.to_excel(writer,'pairs-features')
pairs_features_rr_df.to_excel(writer,'pairs-features-rr')
writer.save()